In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/NLP_Project

/content/drive/MyDrive/NLP_Project


In [ ]:
with open('llama_lora_val_3e.txt', 'r', encoding='UTF-8') as f:
    alpaca_val = f.readlines()

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
par = []
sp = []
abc_par = []

for idx, a in enumerate(alpaca_val):
    if 'targetParagraphs' in a:
        par.append(a)
    elif 'Spoiler' in a or 'spoiler' in a:
        #print(alpaca_val[idx])
        sp.append(alpaca_val[idx + 1])
        if 'targetParagraphs' in alpaca_val[idx - 9]:
            abc_par.append(alpaca_val[idx - 10])
        else:
            abc_par.append(alpaca_val[idx - 9])

In [ ]:
alpaca_dict = dict(zip(abc_par, sp))

In [ ]:

with open("gpt-3.5-turbo-eval.json", 'r', encoding='UTF-8') as f:
    gpt_val = json.load(f)

In [ ]:
preds_gpt = []
truth_gpt = []

for k in gpt_val:
    real = gpt_val[k]['real_spoiler']
    pred = gpt_val[k]['gpt']['choices'][0]['message']['content']
    preds_gpt.append(pred)
    truth_gpt.append(real)

In [ ]:
with open('t5_1e_eval.json', 'r', encoding='UTF-8') as f:
    t5_val = json.load(f)

In [ ]:
preds_t5 = []
truth_t5 = []

for k in t5_val:
    #print(truth_t5[k])
    real = t5_val[k]['real_spoiler']
    pred = t5_val[k]['t5']
    preds_t5.append(pred)
    truth_t5.append(real)

In [ ]:
with open('roberta_10_eval_1b.json', 'r', encoding='UTF-8') as f:
    roberta_10_eval = json.load(f)

with open('roberta_10_eval_4b.json', 'r', encoding='UTF-8') as f:
    roberta_4_eval = json.load(f)

In [ ]:
preds_roberta_4 = []
truth_roberta_4 = []

for k in roberta_4_eval:
    real = roberta_4_eval[k]['spoiler']
    pred = roberta_4_eval[k]['roberta']
    preds_roberta_4.append(pred)
    truth_roberta_4.append(real)


preds_roberta_10 = []
truth_roberta_10 = []

for k in roberta_10_eval:
    real = roberta_10_eval[k]['spoiler']
    pred = roberta_10_eval[k]['roberta']
    preds_roberta_10.append(pred)
    truth_roberta_10.append(real)

In [ ]:
with open('val_2048_10_200.txt', 'r', encoding='UTF-8') as f:
    falcon_val = f.readlines()

In [ ]:
par_falcon = []
par_falcon = []
sp_falcon = []
abc_par_falcon = []

for idx, a in enumerate(falcon_val):
    if 'targetParagraphs' in a:
        par_falcon.append(a)
    elif 'Spoiler' in a or 'spoiler' in a:
        #print(alpaca_val[idx])
        sp_falcon.append(falcon_val[idx])
        if 'targetParagraphs' in falcon_val[idx - 9]:
            abc_par_falcon.append(falcon_val[idx - 10])
        else:
            abc_par_falcon.append(falcon_val[idx - 9])

In [ ]:
ct = 0
sppp = []
cts = []

for i in " ".join(falcon_val).split('$$$'):
    if 'postText' in i:
        cts.append(ct)
        sppp.append(i.split('spoiler:')[1].split('\n')[0])
        ct += 1

In [ ]:
falcon_eval = sppp

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
#nltk.download('punkt')

def calculate_bleu_score(candidate, reference):
    candidate_tokens = nltk.word_tokenize(candidate)
    reference_tokens = nltk.word_tokenize(reference)

    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)

    return bleu_score


with open('validation_changed.json', 'r') as json_file:
    json_list = list(json_file)

val_set = []
targetParagraphs = []
spoilers = []
for json_str in json_list:
    result = json.loads(json_str)
    #val_set.append(result)
    targetParagraphs.append(result['targetParagraphs'])
    spoilers.append(result['spoiler'])

'def calc_score_from_preds(preds, true_data):\n    scorelist = []\n    for idx, spoiler in enumerate(preds):\n        #print(idx)\n        truth = true_data[idx]\n        score = calculate_bleu_score(spoiler, truth)\n        scorelist.append(score)\n    return scorelist'

In [ ]:
pip install bert-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 129.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.9 MB/s eta 0:00:00


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import string
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, f1_score
import json
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from bert_score import score

In [ ]:
def bleu_score(truth, prediction):

    def stopfilter(tokens):
        tmp = [token for token in tokens if token not in stopwords.words('english')]
        res = [token.lower() for token in tmp if token not in string.punctuation]
        return res

    def make_score(trut, predi):
        if len(trut) > 3 and len(predi) > 3:
            weights = (1./4., 1./4., 1./4., 1./4.)
        elif len(trut) > 2 and len(predi) > 2:
            weights = (1./3., 1./3., 1./3.)
        elif len(trut) > 1 and len(predi) > 1:
            weights = (1./2., 1./2.)
        else:
            weights = (1., 0.)

        if (len(weights) == 4) and (len(trut) < 4 or len(predi) < 4):
            print(trut)
            print(predi)
            print(weights)
            print('\n')

        return sentence_bleu([trut], predi, weights=weights)

    score = 0.
    lem_score = 0.

    write_dict = {'single_scores': {}, 'scores': {}}

    for i in range(len(prediction)):
        real_answer = truth[i]
        if type(real_answer) is list:
            real_answer = ' '.join(real_answer)

        pred_answer = prediction[i]
        if type(pred_answer) is list:
            pred_answer = ' '.join(pred_answer)

        lem_truth_tokens = stopfilter(word_tokenize(real_answer.replace('\n', '')))
        lem_prediction_tokens = stopfilter(word_tokenize(pred_answer.replace('\n', '')))
        i_lem_score = make_score(lem_truth_tokens, lem_prediction_tokens)
        lem_score += i_lem_score

    return lem_score / len(truth)


def bert_score(truth, prediction):
    assert len(truth) == len(prediction)
    prec, rec, f1 = score(prediction, truth, lang="en")

    return float(f1.mean())

In [ ]:
def calc_score_from_preds(preds, true_data):
    bleu_scorelist = []
    bert_scorelist = bert_score(true_data, preds)
    for idx, y_pred in enumerate(preds):
        y_true = true_data[idx]

        bleu_sc = bleu_score([y_true], [y_pred])

        bleu_scorelist.append(bleu_sc)

    return np.mean(bleu_scorelist), bert_scorelist

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
scores_alpaca = calc_score_from_preds(sp, spoilers)

In [ ]:
scores_roberta_4 = calc_score_from_preds(preds_roberta_4 , truth_roberta_4)


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
scores_roberta_4

(2.941103940016657e-157, 0.7927424907684326)

In [ ]:

scores_roberta_10 = calc_score_from_preds(preds_roberta_10 , truth_roberta_10)


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
scores_roberta_10

(2.941103940016657e-157, 0.7928134202957153)

In [ ]:
scores_falcon = calc_score_from_preds(falcon_eval , spoilers[:len(falcon_eval)])

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
scores_falcon

(0.011897712891263328, 0.7621487379074097)

In [ ]:
scores_t5 = calc_score_from_preds(preds_t5 , truth_t5)


In [ ]:
scores_gpt = calc_score_from_preds(preds_gpt, truth_gpt)

In [ ]:
scores_alpaca

In [ ]:
scores_roberta_4

In [ ]:
scores_roberta_10

In [ ]:
scores_falcon

In [ ]:
np.mean(scores_t5[0])

0.19242533309076365

In [ ]:
bert_sc = bert_score(truth_t5, preds_t5)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bert_sc

0.8644362092018127

In [ ]:
scores_gpt

In [ ]:
np.mean(scores_gpt[0])

0.14789007026858195

In [ ]:
bert_sc_gpt = bert_score(truth_gpt, preds_gpt)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bert_sc_gpt

0.8666253089904785

In [ ]:
import pickle

with open('my_list.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
t = []
p = []
for d in data:
  p.append(d[0])
  t.append(d[1])

In [ ]:
calc_score_from_preds(p, t)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(0.01835092686884588, 0.7890071868896484)